## Análisis exploratorio de los datos (EDA).

### Descripción de las dimensiones
- id: Identificador del anuncio. 
- url: Link web del anuncio.
- region: Región de Estados Unidos en donde se encuentra la propiedad.
- region_url: Link web de los anuncios pertenecientes a la región. 
- price: Precio de la propiedad en petrodólares.
- type: Tipo de propiedad.
- sqfeet: Metros cuadrados de la propiedad.
- beds: Cantidad de dormitorios.
- baths: Cantidad de baños.
- cats_allowed: Si se permiten gatos en la propiedad toma el valor 1, 0 para caso contrario.
- dogs_allowed: Si se permiten perros en la propiedad toma el valor 1, 0 para caso contrario.
- smoking_allowed: Si se permite fumar en la propiedad toma el valor 1, 0 para caso contrario.
- wheelchair_access: Si la propiedad posee acceso para sillas de ruedas toma el valor 1, 0 para caso contrario.
- electric_vehicle_charge: Si la propiedad posee cargador para vehículos eléctricos toma el valor 1, 0 para caso contrario.
- comes_furnished: Si la propiedad viene amueblada toma el valor 1, 0 para caso contrario.
- laundry_options: Opciones de lavandería (w/d in unit: Lavadora/secadora en la propiedad, w/d hookups: conexión para lavadora/secadora, laundry on site: servicio de lavandería en el lugar, laundry in bldg: servicio de lavandería en el edificio, no laundry on sit: sin servicio de lavandería).
- parking_options: Opciones de estacionamiento (off-street parking: zona de estacionamiento, attached garage: garaje incluido, carport: cochera/garaje abierto, detached garage: garaje separado, street parking: estacionamiento delimitado en la calle, no parking: sin estacionamiento, valet parking: estacionamiento con servicio valet).
- image_url: Link web de la imagen de la propiedad en el anuncio. 
- description: Descripción de la propiedad puesta en el anuncio. 
- lat: Latitud.
- long: Longitud.
- state: Código del estado al que pertenece la propiedad.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


In [ ]:
#Comenzamos leyendo los archivos y viendo los campos que tiene
properties_train = pd.read_parquet('datasets/train.parquet')
properties_test = pd.read_parquet('datasets/test.parquet')

In [ ]:
#Analizamos los primeros 5 registros
properties_train.head(5)

In [ ]:
#Validamos tipos de datos y sus faltantes
properties_train.info()
properties_train.isna().sum()

In [ ]:
#Observamos algunas estadisticas 
properties_train.describe()

In [ ]:
#Validamos si tenemos duplicados en general
properties_train.duplicated().sum()

In [ ]:
#Validamos duplicados de propiedades publicada varias veces con las mismas fotos y descripción 
properties_train.description.duplicated().sum()
properties_train.image_url.duplicated().sum()

### Quitamos duplicados en image_url & description

In [ ]:
#Comparamos 1 de esos duplicados para detectar cual es su variante y detectamos que es el precio, por lo que se va a mantener el último
mask = properties_train["image_url"].str.contains("https://images.craigslist.org/00R0R_5XAoSKvfrz")
properties_train[mask].head()


In [ ]:
properties_train = properties_train.drop_duplicates(subset='image_url', keep='last')
properties_train.shape

### Creamos variable objetivo

In [ ]:
properties_train["category_price"] = properties_train['price'].where(properties_train['price'] < 1000, 0).where(properties_train['price'] >= 1000, 1)

properties_train.head(1)

### Analizamos el mapa, para validar si la lat y long se encuentran ok o tenemos outliers

In [ ]:
import geopandas as gpd
from shapely.geometry import Point

In [ ]:
# Creamos una columna de tipo Point utilizando las columnas 'latitud' y 'longitud'
mapa = properties_train
mapa['geometry'] = mapa.apply(lambda x: Point(x.long, x.lat), axis=1)

# Creamos una columna de tipo GeoSeries utilizando la columna de Point
mapa = gpd.GeoDataFrame(mapa, geometry='geometry')

# Asignar un sistema de coordenadas específico (en este caso, EPSG: 4326)
mapa.crs = {'init': 'epsg:4326'}

In [ ]:
# Graficamos el mapa
mapa.plot()
plt.show()

Podemos ver una dispersión que inncluye otros paises, no solo de EEUU. 
Vamos a analizar si corresponden a los límites de EEUU con un poligono. El mismo tiene un margen de error para las ciudades limitrofes. Descargamos el geo mapa de https://exploratory.io/map

In [ ]:
# importar un poligono de Estados Unidos
usa = gpd.read_file("./datasets/us.geojson")

mapa = properties_train
#Crear un GeoDataFrame a partir de tu dataframe
geo_df = gpd.GeoDataFrame(mapa, geometry=gpd.points_from_xy(mapa.long, mapa.lat))
geo_df.crs = {'init': 'epsg:4326'}

# Hacer una intersección entre el GeoDataFrame y el poligono de USA
result = gpd.sjoin(geo_df, usa, op='within')

# Crear una nueva columna en el dataframe original con un booleano si la propiedad esta dentro de Estados Unidos o no
mapa['in_usa'] = mapa.index.isin(result.index)

In [ ]:
mapa.in_usa.value_counts()
maska = mapa["in_usa"] == False
mapa[maska].region

Podemos ver que descarto varias ciudades que si concuerdan con el pais de EEUU, por lo que vamos a validarlo con un listado de .cvs.
Descargamos el csv con las regiones de la página: https://catalog.data.gov/dataset/?q=list+of+cities&sort=views_recent+desc&ext_location=&ext_bbox=&ext_prev_extent=-150.46875%2C-80.17871349622823%2C151.875%2C80.17871349622823

In [ ]:
cities = pd.read_csv("datasets/cities_usa.csv")
#Comparamos las ciudades excluidas
cities['region'] = cities['region'].str.lower()
mapa['region'] = mapa['region'].str.lower()                                       

mapa['valid_region'] = mapa['region'].isin(cities['region'])

maska = (mapa["valid_region"] == False) & (mapa["in_usa"] == False)

mapa[maska].region.unique()

Validamos que las regiones que no entran en los criterios siguen perteneciendo a EEUU pero con alguna nomenclatura que no contemplamos como la palabra 'city' o las abreviaciones. No se puede descartar ninguna región por el momento.

## Outliers

Validamos outliers con 3 sigmas y zscore de price

In [ ]:
# Vemos distribución de price con outlier
plt.boxplot(properties_train["price"], vert=False)

# Mostrar el gráfico
plt.show()

In [ ]:
# Media y la desviación estándar
mean = properties_train["price"].mean()
std = properties_train["price"].std()
outliers_3 = properties_train.query("(price < (@mean - 3 * @std)) | (price > (@mean + 3 * @std))")
outliers_3


# Calcular el Z-score
properties_train["zscore"] = (properties_train["price"] - properties_train["price"].mean()) / properties_train["price"].std()
outliers_s = properties_train[(properties_train["zscore"] > 3) | (properties_train["zscore"] < -3)]
outliers_s

In [ ]:
#Eliminamos outlier identificado para ver si modifica el gráfico

properties_train = properties_train.drop(outliers_3.index)
# Crear el gráfico de caja
plt.boxplot(properties_train["price"], vert=False)

# Mostrar el gráfico
plt.show()

Outlier de sqfeet

In [ ]:
# Vemos distribución de sqfeet con outlier
plt.boxplot(properties_train["sqfeet"], vert=False)

# Mostrar el gráfico
plt.show()

In [ ]:
#Validamos 3 sigmas y Z-core

# 3 sigmas
mean = properties_train["sqfeet"].mean()
std = properties_train["sqfeet"].std()
outliers_sq3 = properties_train.query("(sqfeet < (@mean - 3 * @std)) | (sqfeet > (@mean + 3 * @std))")
outliers_sq3

# Z-score
properties_train["zscore"] = (properties_train["sqfeet"] - properties_train["sqfeet"].mean()) / properties_train["sqfeet"].std()
outliers_b = properties_train[(properties_train["zscore"] > 3) | (properties_train["zscore"] < -3)]
outliers_b


In [ ]:
properties_train = properties_train.drop(outliers_sq3.index)

# Crear el gráfico de caja
plt.boxplot(properties_train["sqfeet"], vert=False)

# Mostrar el gráfico
plt.show()

Outliers de beds

In [ ]:
#No considero que tengan un outlier

properties_train.baths.unique()

properties_train.cats_allowed.unique()

properties_train.dogs_allowed.unique()

properties_train.smoking_allowed.unique()

properties_train.wheelchair_access.unique()

properties_train.electric_vehicle_charge.unique()

properties_train.comes_furnished.unique()


In [ ]:
properties_train.beds.unique()
#Claramente tenemos valores outliers

In [ ]:
# Vemos distribución de sqfeet con outlier
plt.boxplot(properties_train["beds"], vert=False)

# Mostrar el gráfico
plt.show()

In [ ]:
maskara=properties_train.beds > 10
properties_train[maskara]

#eliminamos los datos outliers con máscara
properties_train = properties_train.drop(properties_train[maskara].index)

In [ ]:
# Vemos distribución de sqfeet con outlier
plt.boxplot(properties_train["beds"], vert=False)

# Mostrar el gráfico
plt.show()

In [ ]:
#Eliminamos columnas que no suman
properties_train.drop(['geometry', 'in_usa', 'valid_region','zscore','url', 'region_url','image_url','description'], axis=1, inplace=True)

In [ ]:
properties_train.columns

columnas = ['sqfeet', 'beds', 'baths',
       'cats_allowed', 'dogs_allowed', 'smoking_allowed', 'wheelchair_access',
       'electric_vehicle_charge', 'comes_furnished', 'category_price']

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(properties_train[columnas].corr(method='spearman'), annot=True, fmt='.1g', cmap='coolwarm')

### Revisamos nuevamente nulos y separamos dataframes

In [ ]:
#Notamos que no hubo un cambio quitando los duplicados de image_url
properties_train.isna().sum()

In [ ]:
#Debido a que: laundry_options y parking_options representan un 19% de los datos vamos a validar si realmente tiene impacto en el precio.
properties_train.laundry_options.unique()
properties_train.parking_options.unique()

# Gráfico de barras de category_price segun laundry
plot = pd.crosstab(index=properties_train['laundry_options'],
            columns=properties_train['category_price']).apply(lambda r: r/r.sum() *100,
                                              axis=1).plot(kind='bar')



In [ ]:
# Gráfico de barras de category_price segun parking
plot = pd.crosstab(index=properties_train['parking_options'],
            columns=properties_train['category_price']).apply(lambda r: r/r.sum() *100,
                                              axis=1).plot(kind='bar')

Observación:
Por lo que se observa a simple vista, las categorias tienen un impacto en el precio de la propiedad. Es decir que aumenta el precio por tener estacionamiento y tener un lavadero en la propiedad también. 

## Archivo remplazando nulos - properties_train_rep

Vamos a completar con valores desconocidos y vemos como impacta en precio

In [ ]:
properties_train_rep = properties_train.fillna({'laundry_options': 'Unknown'})
properties_train_rep = properties_train.fillna({'parking_options': 'Unknown'})
properties_train_rep['laundry_options'] = properties_train['laundry_options'].replace(np.nan, 'Unknown')
properties_train_rep.isna().sum()

Los demás datos nulos de lat y long representan el 5% de la información por lo que vamos a eliminarlos. 

In [ ]:
#Vamos a crear 2 dataframe, 1 sin nulos y 1 con nulos para analizar los resultados con ambos

properties_train_rep = properties_train_rep.dropna()
properties_train_rep.shape

In [ ]:
properties_train.columns

In [ ]:
#Creamos archivo de properties con remplazo para comenzar a hacer un modelo en otra notebook. 

properties_train_rep.to_csv('datasets/properties_train_rep.csv', index=True) 

## Archivo remplazando nulos - properties_train_wn

In [ ]:
properties_train.isna().sum()

In [ ]:
properties_train_wn = properties_train.dropna()
properties_train_wn.isna().sum()

In [ ]:
properties_train_wn.shape

In [ ]:
#Creamos archivo de properties sin nulos para comenzar a hacer un modelo en otra notebook. 

properties_train_rep.to_csv('datasets/properties_train_wn.csv', index=True) 